## Example notebook for loading MR Images

Download a MR Image sample from physionet.org
* https://physionet.org/content/images/1.0.0/

alternatively also CT image from DICOM library
* https://www.dicomlibrary.com/


In [1]:
import os
import urllib.request

url = "https://physionet.org/files/images/1.0.0/E1154S7I.dcm"
file_name = "E1154S7I.dcm"
data_path = "./data"

def download_file(root_dir: str, url: str, file_name: str):
    # try to make a root dir if not exists
    os.makedirs(name=root_dir, exist_ok=True)
    file_path = os.path.join(root_dir, file_name)

    if not os.path.exists(file_path):
        # url must contain the file anme
        # https://stackoverflow.com/questions/46413651/download-file-with-urlretrieve-to-subfolder
        urllib.request.urlretrieve(url, file_path)
    else:
        print(f"file {file_name} already exists in {file_path}")


download_file(data_path, url, file_name)

file E1154S7I.dcm already exists in ./data/E1154S7I.dcm


In [2]:
import pandas as pd
from pathlib import Path
import SimpleITK as sitk
from pydicom import read_file
import numpy as np

In [3]:
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 1000)
# pd.set_option('display.max_colwidth', 1000)

In [4]:
from __future__ import print_function
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


In [5]:
from pydicom import read_file, dcmread
from pathlib import Path
import pandas as pd
from tqdm import tqdm

def get_meta(meta_info, key):
    instance_name = [el.keyword for el in meta_info if key.lower() in str(el.keyword).lower()]
    if 0<len(instance_name)<=1:
        return getattr(meta_info, instance_name[0])
    elif len(instance_name)>1:
        print ("Multiple keys found: ", instance_name)
        values =  np.unique(np.array([getattr(meta_info, el) for el in instance_name]))
        if len(values) >1:
            print("Multiple instances for these keys: ", values)
            return None
        else:
            return values[0]
    else:
        print("No key found")
        return None
    
def try_open_dcm(pydicom_image, sitk_image, dc_path):

    try:
        oct_volume = np.array(pydicom_image.pixel_array)
        flag_pydicom_ok = True
    except:
        flag_pydicom_ok = False

    
    try:
        sitk_image.ReadImageInformation()
        image = sitk.ReadImage(str(dc_path))
        flag_sitk_ok = True
    except:
        flag_sitk_ok = False

    return flag_pydicom_ok, flag_sitk_ok


def get_all_dcm_meta(input_path):
    dc_paths= [dc_path for dc_path in Path(input_path).rglob('*.dcm')]


    df = pd.DataFrame()
    for dc_path in tqdm(dc_paths):

        sitk_image_reader = sitk.ImageFileReader()
        # only read DICOM images
        sitk_image_reader.SetImageIO('GDCMImageIO')
        sitk_image_reader.SetFileName(str(dc_path))
        pydicom_reader = read_file(str(dc_path))
        row_dict ={el.keyword: getattr(pydicom_reader, el.keyword) for el in pydicom_reader if el.keyword!= "" and el.keyword!="PixelData"}
        
        # Reading patient info
        patient_doctor_id = "".join(pydicom_reader.PatientName)
        # row_dict["patient_id"] = patient_doctor_id.split("^")[0]
        # row_dict["doctor_id"] = patient_doctor_id.split("^")[1]
        
        
        flag_pydicom_ok, flag_sitk_ok = try_open_dcm(pydicom_reader, sitk_image_reader, dc_path)
        row_dict["pydicom_readable"] = flag_pydicom_ok
        row_dict["sitk_readable"] = flag_sitk_ok
        row_dict["oct_readable"] = (flag_sitk_ok or flag_pydicom_ok)

        if flag_pydicom_ok or flag_sitk_ok:
            print(dc_path.name, flag_pydicom_ok, flag_sitk_ok)


            sitk_image_reader.ReadImageInformation()
            image = sitk.ReadImage(str(dc_path))
            depth = image.GetDepth()
            row_dict["num_slices"]= depth
            row_dict["img_size"]= image.GetSize()

        row_dict["filename"] = str(Path(dc_path).name)

        
        row = pd.DataFrame.from_records([row_dict])

        df = pd.concat([df, row], ignore_index=True)


    return df


In [6]:
OCT_EXPORT_1 = data_path
df_export_1 = get_all_dcm_meta(OCT_EXPORT_1)

100%|██████████| 1/1 [00:00<00:00,  8.37it/s]

E1154S7I.dcm True True


In [7]:
df_export_1.head()

,ImageType,SOPClassUID,SOPInstanceUID,StudyDate,StudyTime,AccessionNumber,Modality,Manufacturer,StudyDescription,PatientName,PatientID,PatientBirthDate,PatientSex,FrameTime,StudyInstanceUID,SeriesInstanceUID,StudyID,SeriesNumber,InstanceNumber,SamplesPerPixel,PhotometricInterpretation,NumberOfFrames,FrameIncrementPointer,Rows,Columns,BitsAllocated,BitsStored,HighBit,PixelRepresentation,pydicom_readable,sitk_readable,oct_readable,num_slices,img_size,filename
0,,1.2.840.10008.5.1.4.1.1.3.1,999.999.999,,,,MR,,,(),,,,100.0,999.999.999,999.999.999,,None,None,1,MONOCHROME2,76,1577059,512,512,16,16,0,0,True,True,True,76,"(512, 512, 76)",E1154S7I.dcm


In [8]:
df_export_1[["Modality", "img_size", "filename"]].head()

,Modality,img_size,filename
0,MR,"(512, 512, 76)",E1154S7I.dcm


In [9]:
# Select the first MR images
# t = df_export_1[df_export_1.Modality=="OPT"]
t = df_export_1[df_export_1.Modality=="MR"]

In [10]:
t

,ImageType,SOPClassUID,SOPInstanceUID,StudyDate,StudyTime,AccessionNumber,Modality,Manufacturer,StudyDescription,PatientName,PatientID,PatientBirthDate,PatientSex,FrameTime,StudyInstanceUID,SeriesInstanceUID,StudyID,SeriesNumber,InstanceNumber,SamplesPerPixel,PhotometricInterpretation,NumberOfFrames,FrameIncrementPointer,Rows,Columns,BitsAllocated,BitsStored,HighBit,PixelRepresentation,pydicom_readable,sitk_readable,oct_readable,num_slices,img_size,filename
0,,1.2.840.10008.5.1.4.1.1.3.1,999.999.999,,,,MR,,,(),,,,100.0,999.999.999,999.999.999,,None,None,1,MONOCHROME2,76,1577059,512,512,16,16,0,0,True,True,True,76,"(512, 512, 76)",E1154S7I.dcm


In [11]:
from __future__ import print_function
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets
from pathlib import Path

import matplotlib.pyplot as plt
def show_raw_dcm(image, slice_idx):
    print(image.shape)
    plt.imshow(image[slice_idx], cmap= "gray")
    plt.title(f"{image.shape}")
    plt.show()
    # do not use close otherwise the interactive widget cell will reload
    # plt.close()

In [12]:
dc_path = Path(OCT_EXPORT_1) / t.filename.iloc[0]
pydicom_image = read_file(str(dc_path))
oct_volume = np.array(pydicom_image.pixel_array)

In [13]:
oct_volume.shape, type(oct_volume)

((76, 512, 512), numpy.ndarray)

## Dark mode for ipywidget in VS Code
* https://stackoverflow.com/questions/75429196/vs-code-jupyter-notebook-dark-theme-for-interactive-elements
* https://github.com/microsoft/vscode-jupyter/issues/9403
* https://github.com/microsoft/vscode-jupyter/issues/7161

Note:
```
comments in css code
/*must starts with no space and end with a space */
otherwise the interactive will keep recompiling. and jumping up and down while interacting by the user.
```

In [14]:
%%html
<style>
/*overwrite hard coded write background by vscode for ipywidges */
.cell-output-ipywidget-background {
   background-color: transparent !important;
}

/*set widget foreground text and color of interactive widget to vs dark theme color */
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}
</style>

In [15]:
slider_widget = widgets.IntSlider(min = 0, max = oct_volume.shape[0]-1, step = 1, value = 0)

w = interactive(show_raw_dcm, 
                image = fixed(oct_volume),
                slice_idx = slider_widget,
)

display(w)

interactive(children=(IntSlider(value=0, description='slice_idx', max=75), Output()), _dom_classes=('widget-in…